# Подготовка модели распознавания рукописных букв и цифр

Необходимо решить задачу классификации на основе датасета рукописных символов EMNIST и оформить модель как сервис. Таким образом, решение состоит из следующих шагов:

Подготовка данных, построение, обучение и тестирование модели.
Обёртка готовой модели в сервис, запуск веб-приложения в Docker-контейнере, подготовка репозитория.


In [44]:
#!pip install --upgrade pip
#!pip install tqdm
#!pip install torch torchvision torchinfo
#!pip install -U scikit-image matplotlib numpy
#!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [ ]:
import os
import cv2
import emnist
import pickle
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision.datasets import MNIST
from torchvision.datasets import EMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Compose, Resize, Normalize

from torchinfo import summary
from torchvision import datasets
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [10]:
#датасет уже скачан и находится в директории data/
#dataset = EMNIST('data/', 'balanced', download=False

In [2]:
images_train, labels_train = emnist.extract_training_samples('balanced')
images_test, labels_test = emnist.extract_test_samples('balanced')

#количество семплов в каждом сплите датасета и размер изображений
print(f'Train: {len(images_train)} samples')
print(f'Test: {len(images_test)} samples')
print(f'Image size: {images_train[0].shape}')

In [1]:
# создаем словарь соответствий mapping
with open('emnist-balanced-mapping.txt', 'r') as f:
    lines = f.readlines()

mapping = {int(line.split()[0]): chr(int(line.split()[1])) for line in lines}

images_train, labels_train = emnist.extract_training_samples('balanced')

characters = [mapping[label] for label in labels_train]

char_counts = pd.Series(characters).value_counts()

for label, symbol in mapping.items():
    count = char_counts[symbol] if symbol in char_counts else 0
    print(f"label: {label}, Символ: {symbol}, Кол-во семплов: {count}")

In [ ]:
# примеры изображения
import matplotlib.pyplot as plt

for i in range(5):
    image = images_train[i].reshape(28, 28)
    plt.imshow(image, cmap='gray')
    plt.show()

In [ ]:
# # редактирование ориентации (зеркализации) изображения
# def display_image(image):
#     # Проверка необходимой ориентации
#     if len(image.shape) == 3:
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     plt.imshow(image, cmap='gray')
#     plt.axis('off')
#     plt.show()

# # Загрузка и отображение изображения
# image_path = 'path_to_your_image.png'
# image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Если это черно-белое изображение
# display_image(image)

In [22]:
# mapping сохраняем в ф-л mapping.pkl
with open(os.path.join('myapp', 'mapping.pkl'),'wb') as f:
    pickle.dump(mapping, f)

In [233]:
# переводим в тензоры
transform = Compose([
    ToTensor(),
    #Normalize([0.5], [0.5])
    Normalize([0.1307], [0.3081])
])

train_dataset = datasets.EMNIST('data/', split='balanced', train=True, download=True, transform=transform)
val_dataset = datasets.EMNIST('data/', split='balanced', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256)

In [ ]:
#Conv2d - это слой (layer) в библиотеке PyTorch 2D свёртки
#параметры слоя Conv2d включают:
# in_channels: количество каналов входящего изображения.
# out_channels: количество выходных каналов (то есть количество ядер свёртки).
# kernel_size: размер ядра свёртки (например, 3x3, 5x5 и т.д.).
# stride: шаг свёртки (по умолчанию равен 1).
# padding: тип паддинга (валидный или полный, по умолчанию "валидный").
# dilation: значение разреженности (по умолчанию равно 1).

In [235]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=1,  # свертка на вход приходит ч.б. изображение-1канал  (цветное 3канала)
                      out_channels=32, 
                      kernel_size=3,
                      padding=1),                   
            nn.ReLU(),                  # делаем активацию функцией ReLU
            
            nn.MaxPool2d(kernel_size=2), # MaxPool2d уменьшаем кол-во параметров (кол-во признаков/4) 

            # + слой Conv2d для дилатации dilation=2
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, dilation=2, padding=2),
            nn.ReLU(),

            #nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2),
            #nn.ReLU(),
            #nn.AvgPool2d(kernel_size=3, stride=2),
            
            nn.Flatten(),# вытягиваем в вектор
# предиктор...слой 6272признаков (28х28)х32/4 (MaxPool2d) на вход
            nn.Linear(in_features=6272, out_features=64), # задаем кол-во скрытых признаков out_features=128
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=128), # задаем кол-во скрытых признаков out_features=128
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=n_classes) # предскажем кол-во классов out_features=n_classes net = CNN(47)
         )

    def forward(self,x):
        return self.model(x)

In [236]:
# !!!!!!!!!!!!эту ячейку надо запускать перед предиктором (закоментировать) тогда посчитает кол-во признаков 6272
net = CNN(47)
summary(net, input_size=(1, 1, 28, 28))# 1 - количество батчей (batch size), 1 - количество каналов (channels), 28х28 размер высота ширина
#после повторного запуска видим в конце вектор из 47 предсказаний Linear: 

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [1, 47]                   --
├─Sequential: 1-1                        [1, 47]                   --
│    └─Conv2d: 2-1                       [1, 32, 28, 28]           320
│    └─ReLU: 2-2                         [1, 32, 28, 28]           --
│    └─MaxPool2d: 2-3                    [1, 32, 14, 14]           --
│    └─Conv2d: 2-4                       [1, 32, 14, 14]           9,248
│    └─ReLU: 2-5                         [1, 32, 14, 14]           --
│    └─Flatten: 2-6                      [1, 6272]                 --
│    └─Linear: 2-7                       [1, 64]                   401,472
│    └─ReLU: 2-8                         [1, 64]                   --
│    └─Linear: 2-9                       [1, 128]                  8,320
│    └─ReLU: 2-10                        [1, 128]                  --
│    └─Linear: 2-11                      [1, 47]                   6,063


In [142]:
best_val_accuracy = 0
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        print(f'Epoch: {epoch}')
        for step, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

            if step % 10 == 0:
                print(f'Iter: {step} \tLoss: {loss.item()}')

        print(f'Mean Train Loss: {loss_sum / (step + 1):.6f}', end='\n\n')

        if epoch % val_fre == 0:
            validate(model, val_loader)

    print(f'Best Validation Accuracy: {best_val_accuracy}')

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data).squeeze(1)
            loss = loss_f(output, target)
            loss_sum += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)

    accuracy = correct / total
    print(f'Val Loss: {loss_sum / len(val_loader):.6f} \tValidation Accuracy: {accuracy:.4f}')
    # Обновление лучшей точности, если текущая точность выше
    global best_val_accuracy
    if accuracy > best_val_accuracy:
        best_val_accuracy = accuracy

In [2]:
model = CNN(47)                # создаем модель 47 классов которые надо распознать (mapping) 
loss_f = nn.CrossEntropyLoss() # фиксируем Loss
#loss_f = nn.NLLLoss() #  другие функции потерь
#loss_f =nn.MSELoss()
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-1) # применяем оптимизатор
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-5, amsgrad=True)

n_epoch = 20 # кол-во эпох
val_fre = 2  # как часто делаем валидацию

train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
validate(model, val_loader)
print(f'Accuracy: {best_val_accuracy}')
# ошибки Val Loss: уменьшаются,  точность 	Accuracy: растет - модель обучается 
# (обучение может остановится и даже пойти переобучение Val Loss растет)

In [238]:
print(f'Accuracy: {best_val_accuracy}')

Accuracy: 0.8707446808510638


In [ ]:
# batch_size=1000             Val Loss: 0.471095 	Accuracy: 0.8432978723404255
# batch_size=512              Val Loss: 0.444851 	Accuracy: 0.8554787234042553
# batch_size=512 +1слойLinear_64 Val Loss: 0.441709 Accuracy: 0.8507446808510638
# batch_size=512 + nn.NLLLoss Val Loss: nan 	    Accuracy: 0.02127659574468085
# batch_size=256 features=64  Val Loss: 0.565492 	Accuracy: 0.8465425531914894
# batch_size=512 Normalize([0.1307], [0.3081]) Val Loss: 0.469646 	Accuracy: 0.8529787234042553
# batch_size=512 + слой dilation=2, padding=2       Accuracy: 0.8661170212765957
# batch_size=512 + torch.optim.Adam                 Accuracy: 0.8668617021276596
# batch_size=256 + torch.optim.Adam + Normalize     Accuracy: 0.8698404255319149
# batch_size=256 + torch.optim.Adam + Normalize + Linear Accuracy: 0.8707446808510638!!!

In [ ]:
# создаем директорию для сохранения весов
import os
os.makedirs('checkpoints/', exist_ok=True)

In [ ]:
torch.save(model.state_dict(), 'checkpoints/cnn.pth')# сохраняем веса модели

In [ ]:
model.load_state_dict(torch.load('checkpoints/cnn.pth'))# прикручиваем к ней словарь с весами и смотрим метрики
validate(model, val_loader)

In [ ]:
#модель сохраняем в ф-л model.pkl
with open(os.path.join('myapp', 'model.pkl'),'wb') as f:
    pickle.dump(clf, f)

Вы можете заметить, что рукописные символы распознаются хуже, чем картинки из датасета. При желании попробуйте приблизить их к тем, на которых училась ваша модель. Возможно, вам пригодятся фильтры: Eroding and Dilating и Smoothing Images.